In [11]:

import queue

from lxml import html
from selenium.webdriver import Chrome, Edge
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def GetChromeBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    except Exception as err:
        print(err)


def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")

    options.add_experimental_option("excludeSwitches", ['enable-automation'])
    options.add_experimental_option("useAutomationExtension", False)

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--mute-audio")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-gpu")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)


resultQ = queue.Queue()
errorQ = queue.Queue()
totalCount = 1377 * 15


def Scrapper(driver):
    global totalCount
    pageId = 1377
    totalGptsXpath = '//p[contains(text(), "We have found")]'
    linkXpath = '//a[@class="link card-title w-48"]/@href'

    while True:
        next = f"//a[contains(@href, 'page={pageId + 1}')]"
        try:
            cc_element = WebDriverWait(driver, 5).until(
                EC.visibility_of_all_elements_located((By.XPATH, totalGptsXpath)))
        except:
            pass

        try:
            cc_element = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, linkXpath)))
        except Exception as e:
            pass

        page_source = driver.page_source
        doc = html.fromstring(page_source)

        urls = doc.xpath(linkXpath)
        count = int(''.join(doc.xpath('//p[contains(text(), "We have found")]/text()')) \
                    .replace('We have found', '') \
                    .replace('GPTs', '').strip())

        totalCount += len(urls)
        with open(f'pages/{pageId}.txt', 'wt') as file:
            file.writelines('\n'.join(urls))

        if totalCount < count:
            try:
                cc_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, next)))
                cc_element.click()
                pageId += 1
            except Exception as e:
                pass
        else:
            break


url = f'https://gptstore.ai/gpts?page=1377'
driver = GetEdgeBrowser(False)
driver.get(url)
Scrapper(driver)


In [ ]:


title = '//h1/text()'
description = '//p[@class="leading-6 text-gray-500"]/text()'

welcome_msg = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Welcome message")]/following-sibling::dd/descendant::text()'

author = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Author")]/following-sibling::dd/descendant::span/text()'
href = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Author")]/following-sibling::dd/descendant::a/@href'

features = '//dt[contains(@class, "text-xl font-medium leading-6") and h2[contains(text(), "Features and Functions")]]/following-sibling::dd/descendant::span[contains(@class,"font-medium")]/descendant::text()'

prompts = '//dt[contains(@class, "text-xl font-medium leading-6") and h2[contains(text(), "Prompt Starters")]]/following-sibling::dd/descendant::span[contains(@class,"font-medium")]/descendant::text()'

related = '//ul[contains(@class, "grid gap-2 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-3")]/li'

for i in []:
    url = i.xpath('descendant::a/@href')
    title = i.xpath('descendant::a/text()')
    desc = i.xpath('descendant::p/text()')


def Scrapper(driver,url):

    while True:
        next = f"//a[contains(@href, 'page={pageId + 1}')]"
        try:
            cc_element = WebDriverWait(driver, 5).until(
                EC.visibility_of_all_elements_located((By.XPATH, totalGptsXpath)))
        except:
            pass

        try:
            cc_element = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, linkXpath)))
        except Exception as e:
            pass

        page_source = driver.page_source
        doc = html.fromstring(page_source)

        urls = doc.xpath(linkXpath)
        count = int(''.join(doc.xpath('//p[contains(text(), "We have found")]/text()')) \
                    .replace('We have found', '') \
                    .replace('GPTs', '').strip())

        totalCount += len(urls)
        with open(f'pages/{pageId}.txt', 'wt') as file:
            file.writelines('\n'.join(urls))

        if totalCount < count:
            try:
                cc_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, next)))
                cc_element.click()
                pageId += 1
            except Exception as e:
                pass
        else:
            break
